In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import datetime
from IPython.display import display, Markdown
from pathvalidate import sanitize_filename

import utils

from prompts import Job_Post, Resume_Builder

In [3]:
## Inputs
my_files_dir = "my_applications"  # location for all job and resume files
job_file = "job.txt"  # filename with job post text. The entire job post can be pasted in this file, as is.
raw_resume_file = "resume_raw.yaml"  # filename for raw resume yaml. See example in repo for instructions.

# Model for matching resume to job post (except extraction chains)
# gpt-4 is not publicly available yet and can be 20-30 times costlier than the default model gpt-3.5-turbo.
# Simple extraction chains are hardcoded to use the cheaper gpt-3.5 model.
openai_model_name = "gpt-4"
openai_model_name = "gpt-3.5-turbo"

# temperature lower than 1 is preferred. temperature=0 returns deterministic output
temperature = 0.5

llm_kwargs = dict(
    model_name=openai_model_name,
    model_kwargs=dict(top_p=0.6, frequency_penalty=0.1),
)

In [4]:
# Step 1 - Read and parse job posting
job_post = Job_Post(
    utils.read_jobfile(os.path.join(my_files_dir, job_file)),
)
parsed_job = job_post.parse_job_post(verbose=False)

company_name = parsed_job["company"]
job_title = parsed_job["job_title"]
today_date = datetime.today().strftime("%Y%m%d")
job_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
print(f"#filename: {job_filename}.job\n")
utils.write_yaml(parsed_job, filename=f"{job_filename}.job")

#filename: my_applications\20230915__Noir__VueJS Developer.job



In [5]:
# Step 2 - read raw resume and create Resume builder object
my_resume = Resume_Builder(
    resume=utils.read_yaml(filename=os.path.join(my_files_dir, raw_resume_file)),
    parsed_job=parsed_job,
    llm_kwargs=llm_kwargs,
)
projects = my_resume.projects_raw
experiences = my_resume.experiences_raw
skills = my_resume.skills_raw

In [6]:
# Step 3 - Rephrase unedited experiences
experiences = my_resume.rewrite_unedited_experiences(verbose=False)
utils.write_yaml(dict(experiences=experiences))

SHANGHAI FRIENDESS ELECTRONICS TECHNOLOGY CO., LTD. Shanghai, China Start:
SHANGHAI FRIENDESS ELECTRONICS TECHNOLOGY CO., LTD. Shanghai, China End:
TUQI CONSTRUCTION TECHONOLOGY CO., LTD. Shanghai, China Start:
TUQI CONSTRUCTION TECHONOLOGY CO., LTD. Shanghai, China End:
Jinhua Toast Network Technology Co., Ltd. Changchun, China Start:
Jinhua Toast Network Technology Co., Ltd. Changchun, China End:


In [10]:
# Step 4 - Rephrase projects
my_resume.experiences = experiences

projects = my_resume.rewrite_projects_desc(verbose=False)
utils.write_yaml(dict(projects=projects))

In [11]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits2 = utils.read_yaml(edits)
    if "experiences" in edits2:
        experiences = edits2["experiences"]
    if "projects" in edits2:
        projects = edits2["projects"]
    if "skills" in edits2:
        skills = edits2["skills"]
    if "summary" in edits2:
        summary = edits2["summary"]

In [12]:
# Step 5 - Extract skills
# This will match the required skills from the job post with your resume sections
# Outputs a combined list of skills extracted from the job post and included in the raw resume
my_resume.experiences = experiences
my_resume.projects = projects

skills = my_resume.extract_matched_skills(verbose=False)
utils.write_yaml(dict(skills=skills))

In [13]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits3 = utils.read_yaml(edits)
    if "experiences" in edits3:
        experiences = edits3["experiences"]
    if "projects" in edits3:
        projects = edits3["projects"]
    if "skills" in edits3:
        skills = edits3["skills"]
    if "summary" in edits3:
        summary = edits3["summary"]

In [14]:
# Step 6 - Create a resume summary
my_resume.experiences = experiences
my_resume.skills = skills
my_resume.projects = projects

summary = my_resume.write_summary(
    verbose=True,
)
utils.write_yaml(dict(summary=summary))



> Entering new  chain...
Prompt after formatting:
System: You are an expert technical writer. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
Noir
VueJS Developer position at a global record label in London, working on critical projects for their new online music shopping channel. Flexible working hours, home working, and amazing offices.
Human: <Resume>
Education:
- Master's in Computer Science - Augment and Virtual Reality
- Bachelor's in Information management and information system

Experience:
- 

- 1 years experience in:
  - Developed web-based features using VueJS and Typescript for customized reports with 200,000+ users/month.
  - Developed a material calculation system using VueJS and Flask, reducing production time by up to 30%.
  - Created a user-friendly 3D house layout model generation system using VueJS and BabylonJS.

- 0 years experience in:
  - Developed a real-time 3D display of a complex environment u

In [17]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed.
edits = """ """

edits = edits.strip()
if edits:
    edits4 = utils.read_yaml(edits)
    if "experiences" in edits4:
        experiences = edits4["experiences"]
    if "projects" in edits4:
        projects = edits4["projects"]
    if "skills" in edits4:
        skills = edits4["skills"]
    if "summary" in edits4:
        summary = edits4["summary"]

In [18]:
# Step 7 - Generate final resume yaml for review
my_resume.summary = summary
my_resume.experiences = experiences
my_resume.projects = projects
my_resume.skills = skills

today_date = datetime.today().strftime("%Y%m%d")
resume_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
resume_final = my_resume.finalize()
print(f"#filename: {resume_filename}.yaml\n")
utils.write_yaml(resume_final, filename=f"{resume_filename}.yaml")

#filename: my_applications\20230915__Noir__VueJS Developer.yaml



In [19]:
# Step 8 - Identify resume improvements
# A previously generated resume can also be used here by manually providing resume_filename. Requires the associated parsed job file.
final_resume = Resume_Builder(
    resume=utils.read_yaml(filename=f"{resume_filename}.yaml"),
    parsed_job=utils.read_yaml(filename=f"{resume_filename}.job"),
    is_final=True,
    llm_kwargs=llm_kwargs,
)
improvements = final_resume.suggest_improvements(verbose=True)
utils.write_yaml(dict(improvements=improvements))



> Entering new  chain...
Prompt after formatting:
System: You are an expert critic. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
The ideal candidate is able to perform the following duties:
- Developing web applications using VueJS, HTML5, CSS3, TypeScript, GIT Version Control, JavaScript, SASS, jQuery, React, and AJAX
- Working on projects critical to the launch of the new online music shopping channel
- Collaborating with the team to deliver high-quality code and user experiences
- Participating in code reviews and providing constructive feedback
- Keeping up to date with industry trends and best practices

The ideal candidate has the following qualifications:
- Experience with VueJS, HTML5, CSS3, TypeScript, GIT Version Control, JavaScript, SASS, jQuery, React, and AJAX
- Strong problem-solving and debugging skills
- Ability to work independently and in a team
- Excellent communication and collaboration skills
- P


> Finished chain.
Chain output:
<Plan>
1. Review the <Resume> for spelling and grammar errors.
2. Identify areas in the <Resume> that can be improved to better align with the <Job Posting>.
3. Provide specific suggestions for improvement in each section of the <Resume>.
4. Compile all suggestions into the <Final Answer>.
5. Check for any additional spelling and grammar errors in the <Final Answer>.

<Additional Steps>
- Ensure that the suggestions provided are clear and actionable.
- Make sure to explain how each suggestion will help showcase that the candidate meets the requirements of the <Job Posting>.

<Work>
Spelling and Grammar:
- No spelling or grammar errors were found in the <Resume>.

Summary:
- The summary effectively highlights the candidate's experience with VueJS and their specialization in Augment and Virtual Reality. However, it could be improved by mentioning their experience with other technologies mentioned in the <Job Posting>, such as React and TypeScript.

Experi

In [6]:
# Step 9 - Generate pdf from yaml
# Most common errors during pdf generation occur due to special characters. Escape them with backslashes in the yaml, e.g. $ -> \$
# pdf_file = utils.generate_pdf(yaml_file=f"{resume_filename}.yaml")
# display(Markdown((f"[{pdf_file}](<{pdf_file}>)")))

utils.generate_new_tex(yaml_file="my_applications/20230915__NOIR__VueJS Developer.yaml")

Done
